### This notebook is to explore and develop functionality to output our downsampled imagery as COGeoTIFF

In [3]:
import os
import sys
from pathlib import Path

module_path = os.path.abspath(os.path.join('..')) # add the projects src code directory to the sys path to allow finding our module
if module_path not in sys.path:
    sys.path.append(module_path+"\\src\\data")
    #sys.path.insert(0, module_path+"\\src\\data")
    
import downSampler # load pipeline module

#### Lets look at our generated file

In [4]:
input_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_input' # path to input data files
output_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_output' # path to input data files

In [5]:
input_file_dict  = downSampler.find_files(input_data_path)
input_file_dict

Finding data files...


{'NEON_D16_ABBY_DP3_552000_5071000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5071000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5072000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5072000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5073000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5073000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5074000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.

In [6]:
output_file_dict = downSampler.find_files(output_data_path)
output_file_dict

Finding data files...


{'Wyvern_D10_RMNP_DP3_459000_4448000_2017_reflectance_downsampled_4mGSD.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_output\\hdf5_downsampled\\Wyvern_D10_RMNP_DP3_459000_4448000_2017_reflectance_downsampled_4mGSD.h5',
 'Wyvern_D10_RMNP_DP3_459000_4448000_2018_reflectance_downsampled_4mGSD.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_output\\hdf5_downsampled\\Wyvern_D10_RMNP_DP3_459000_4448000_2018_reflectance_downsampled_4mGSD.h5',
 'Wyvern_D16_ABBY_DP3_552000_5071000_reflectance_downsampled_4mGSD.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_output\\hdf5_downsampled\\Wyvern_D16_ABBY_DP3_552000_5071000_reflectance_downsampled_4mGSD.h5',
 'Wyvern_D16_ABBY_DP3_552000_5072000_reflectance_downsampled_4mGSD.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_output\\hdf5_downsampled\\Wyvern_D16_ABBY_DP3_552000_5072000_reflect

#### Check the input data structure

In [7]:
in_keys_ls = list(input_file_dict.keys())
in_keys_ls
# next(iter(input_file_dict)) alternative way
downSampler.h5dump(input_file_dict[in_keys_ls[0]])

	 - ABBY : <HDF5 group "/ABBY" (1 members)>
		 - Reflectance : <HDF5 group "/ABBY/Reflectance" (2 members)>
			 - Metadata : <HDF5 group "/ABBY/Reflectance/Metadata" (7 members)>
				 - Ancillary_Imagery : <HDF5 group "/ABBY/Reflectance/Metadata/Ancillary_Imagery" (14 members)>
					 - Aerosol_Optical_Depth : <HDF5 dataset "Aerosol_Optical_Depth": shape (1000, 1000), type "<i2">
							 - Band_Names : b'AOT (aerosol optical thickness at 550 nm)*1000'
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aerosol Optical Depth at 500 nm.'
							 - Scale_Factor : 1000.0
							 - Units : b'Aerosol Optical Depth at 500 nm.'
					 - Aspect : <HDF5 dataset "Aspect": shape (1000, 1000), type "<f4">
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aspect used as input to ATCOR'
							 - Dimension_Labels : b'-'
							 - Scale_Factor : 1.0
							 - Units : b'degrees'
					 - Cast_Shadow : <HDF5 dataset "Cast_Shadow": shape (1000, 1000), type "|u1">
							 - Data_Ign

In [8]:
out_keys_ls = list(output_file_dict.keys())
out_keys_ls
# next(iter(input_file_dict)) alternative way
downSampler.h5dump(output_file_dict[out_keys_ls[0]])

	 - Reflectance : <HDF5 group "/Reflectance" (2 members)>
		 - Metadata : <HDF5 group "/Reflectance/Metadata" (2 members)>
			 - FWHM : <HDF5 dataset "FWHM": shape (18,), type "<f8">
					 - Description : Full width half maximum of reflectance bands.
					 - Units : nanometers
			 - Wavelength : <HDF5 dataset "Wavelength": shape (18,), type "<f8">
					 - Description : Central wavelength of the reflectance bands.
					 - Units : nanometers
		 - Reflectance_Data : <HDF5 dataset "Reflectance_Data": shape (1000, 1000, 18), type "<i4">
				 - Description : Atmospherically corrected reflectance.
				 - Spatial_Resolution_X_Y : [4. 4.]
				 - data ignore value : -9999.0
				 - reflectance scale factor : 10000.0
				 - spatial extent : [ 459000.  460000. 4448000. 4449000.]


In [12]:
downSampler.h5dump(input_file_dict[in_keys_ls[5]])

	 - RMNP : <HDF5 group "/RMNP" (1 members)>
		 - Reflectance : <HDF5 group "/RMNP/Reflectance" (2 members)>
			 - Metadata : <HDF5 group "/RMNP/Reflectance/Metadata" (7 members)>
				 - Ancillary_Imagery : <HDF5 group "/RMNP/Reflectance/Metadata/Ancillary_Imagery" (14 members)>
					 - Aerosol_Optical_Depth : <HDF5 dataset "Aerosol_Optical_Depth": shape (1000, 1000), type "<i2">
							 - Band_Names : b'AOT (aerosol optical thickness at 550 nm)*1000'
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aerosol Optical Depth at 500 nm.'
							 - Scale_Factor : 1000.0
							 - Units : b'Aerosol Optical Depth at 500 nm.'
					 - Aspect : <HDF5 dataset "Aspect": shape (1000, 1000), type "<f4">
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aspect used as input to ATCOR'
							 - Dimension_Labels : b'-'
							 - Scale_Factor : 1.0
							 - Units : b'degrees'
					 - Cast_Shadow : <HDF5 dataset "Cast_Shadow": shape (1000, 1000), type "|u1">
							 - Data_Ign

In [14]:
refl_array, wavelength_array, FWHM_array, metadata_dict = downSampler.h5data2array(input_file_dict[in_keys_ls[5]])

In [41]:
# work on the output function

def array2h5raster(refl_array, wavelength_array, FWHM_array, metadata_dict, filename_output):
    """
    Takes in a 3-D reflectance array, an array of band centre wavelengths, FWHM array,
    and an additional metadata dictionary and generates a HDF5 file with the given filename.
    """

    # scale the reflectance data up by the original reflectance factor to save disk space
    scale_fac = metadata_dict['reflectance scale factor']
    refl_array = refl_array*scale_fac

    hf = h5py.File(filename_output, 'w') # create hdf5 file
    g1 = hf.create_group('Reflectance') # create main group
    g2 = hf.create_group('Reflectance/Metadata') # group for metadata

    # datasets
    # ------------------------------------------------------------------------------
    # reflectance data
    
    for band in range(len(refl_array[0,0,:])):
        print("band",band)
        refl_dset = g1.create_dataset('Band ' + str(band+1) + ' Reflectance_Data',data=refl_array[:,:,band], dtype='i') # dataset for each band of reflectance data
        refl_dset.attrs['Description'] =  'Atmospherically corrected reflectance.'
        refl_dset.attrs['data ignore value'] = metadata_dict['data ignore value']
        refl_dset.attrs['reflectance scale factor'] = metadata_dict['reflectance scale factor']
        refl_dset.attrs['Spatial_Resolution_X_Y'] = metadata_dict['Spatial_Resolution_X_Y']
        refl_dset.attrs['spatial extent'] = metadata_dict['spatial extent']

    # wavelength data
    wav_dset = g2.create_dataset('Wavelength',data=wavelength_array) # band centre wavelength data
    wav_dset.attrs['Description'] = 'Central wavelength of the reflectance bands.'
    wav_dset.attrs['Units'] = 'nanometers'

    # FWHM data
    FWHM_dset = g2.create_dataset('FWHM',data=FWHM_array) # FWHM data
    FWHM_dset.attrs['Description'] = 'Full width half maximum of reflectance bands.'
    FWHM_dset.attrs['Units'] = 'nanometers'


    hf.close() # close file to save and write to disk


In [32]:
# parameter setup:
data_dir_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_input' # path to input data files

output_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_output' # path to input data files

desired_GSD = 4 # GSD in m

desired_band_centres = [0.505, 0.526, 0.544, 0.565, 0.586, 0.606, 0.626, 0.646, 0.665, 0.682, 0.699, 0.715, 0.730, 0.745, 0.762, 0.779, 0.787, 0.804]

print("Parameters set!")

Parameters set!


In [33]:
# perform downsampling
resamp_refl_array = downSample_reband_array(refl_array, metadata_dict['Spatial_Resolution_X_Y'][0], desired_GSD, wavelength_array, desired_band_centres) # downsample
metadata_dict['Spatial_Resolution_X_Y'] = [float(desired_GSD), float(desired_GSD)] # adjust resolution metadata to reflect downsampling
print("Downsampling Complete...")

Downsampling Complete...


In [37]:
resamp_refl_array.shape

(1000, 1000, 18)

In [42]:
filename_output =  Path(output_data_path / 'TEST_output_1.h5')
band_width_array = downSampler.band_widths(desired_band_centres)
array2h5raster(resamp_refl_array, desired_band_centres, band_width_array, metadata_dict, filename_output)

band 0
band 1
band 2
band 3
band 4
band 5
band 6
band 7
band 8
band 9
band 10
band 11
band 12
band 13
band 14
band 15
band 16
band 17


In [43]:
#Path("C:\Users\Chris\Documents\SideProjects_C\hyperspectral-project\data\pipeline_output\TEST_output_1.h5")
downSampler.h5dump(filename_output)

	 - Reflectance : <HDF5 group "/Reflectance" (19 members)>
		 - Band 1 Reflectance_Data : <HDF5 dataset "Band 1 Reflectance_Data": shape (1000, 1000), type "<i4">
				 - Description : Atmospherically corrected reflectance.
				 - Spatial_Resolution_X_Y : [4. 4.]
				 - data ignore value : -9999.0
				 - reflectance scale factor : 10000.0
				 - spatial extent : [ 459000.  460000. 4448000. 4449000.]
		 - Band 10 Reflectance_Data : <HDF5 dataset "Band 10 Reflectance_Data": shape (1000, 1000), type "<i4">
				 - Description : Atmospherically corrected reflectance.
				 - Spatial_Resolution_X_Y : [4. 4.]
				 - data ignore value : -9999.0
				 - reflectance scale factor : 10000.0
				 - spatial extent : [ 459000.  460000. 4448000. 4449000.]
		 - Band 11 Reflectance_Data : <HDF5 dataset "Band 11 Reflectance_Data": shape (1000, 1000), type "<i4">
				 - Description : Atmospherically corrected reflectance.
				 - Spatial_Resolution_X_Y : [4. 4.]
				 - data ignore value : -9999.0
				 - refle

- needed metadata for georectification

TODO:
- Find out specification of SkyWatches data format, GeoTIFF? COG/COGeoTIFF? (Cloud-optimized GeoTIFF)
- Adapt output code to output to GeoTIFF format
- TEST TEST TEST!

In [25]:
count = 0
for i in range(1, len(refl_array[0,0,:])+1):
    count += 1
    #print(i)
print('count',count)

count 426
